In [63]:
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from datetime import datetime, date

class Element(BaseModel):
    label: str
    lang: Optional[str]
    uri: str
    type: list[str]

class Status(BaseModel):
    value: str = Field(default="mstatus:new")
    label: str = Field(default="novo")

class IdentifiedBy(BaseModel):
    type: str = Field(default='Local')
    assigner: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    value: str  

class GenerationProcess(BaseModel):
    label: str = Field(default="BiblioKeia v.1")
    generationDate: datetime = Field(default=datetime.now())

class AdminMetadata(BaseModel):
    encodingLevel: str = Field(default="menclvl:f")
    assigner: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    creationDate: Optional[str]
    descriptionConventions: str = Field(default="http://id.loc.gov/vocabulary/descriptionConventions/isbd")
    descriptionModifier: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    descriptionLanguage: str = Field(default="http://id.loc.gov/vocabulary/languages/por")
    generationProcess: str = Field(default="BiblioKeia v.1")
    generationDate: datetime = Field(default=datetime.now())
    identifiedBy: list[IdentifiedBy]
    status: Status = Field(default=Status(value="mstatus:new", label="novo"))

    @validator('creationDate', pre=True, always=True)
    def set_date_today(cls, v):
        today = date.today().strftime("%Y-%m-%d")
        
        return today

class Item(BaseModel):
    adminMetadata: AdminMetadata
    call: str
    shelf: str
    barcode: str

class Items_Schema(BaseModel):
  itemOf: Element
  items: List[Item]

In [50]:
from pyfuseki import FusekiUpdate

In [65]:
items = {
  "itemOf": {
      "label": "TITULO INSTNACE",
      'uri': "https://bibliokeia.com/resources/instance/bkc-2",
      'type': ['Instance']
  },
  "items": [
    {
      "adminMetadata": {"identifiedBy": [{"value": "23-8486"}]},
      
      "call": "582.56 O48p 2023",
      "shelf": "E11-P05",
      "barcode": "23-8486"
    }
  ]
}
request = Items_Schema(**items)
request.items

[Item(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='2023-06-27', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 27, 16, 0, 52, 826753), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='23-8486')], status=Status(value='mstatus:new', label='novo')), call='582.56 O48p 2023', shelf='E11-P05', barcode='23-8486')]

In [55]:
request.itemOf

'https://bibliokeia.com/resources/instance/bkc-2'

In [21]:
for item in request.items:
    print(item.call)

582.56 O48p 2023


In [46]:
prefix = """
PREFIX bki: <https://bibliokeia.com/resources/instance/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>  
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
    """

def MakeGraphItems(item, itemOf):
    graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/resources/item/{item.barcode}>
        {{
        <https://bibliokeia.com/resources/item/{item.barcode}> a bf:Item ;
        bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel {item.adminMetadata.encodingLevel} ;
            bf:assigner <{item.adminMetadata.assigner}> ;
            bf:creationDate "{item.adminMetadata.creationDate}"^^xsd:date ;  
            bf:descriptionConventions <{item.adminMetadata.descriptionConventions}> ;
            bf:descriptionLanguage <{item.adminMetadata.descriptionLanguage}> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{item.adminMetadata.generationProcess}" ;
                    bf:generationDate "{item.adminMetadata.generationDate}"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <{item.adminMetadata.assigner}> ;
                    rdf:value "{item.barcode}" ] ;
            bf:status {item.adminMetadata.status.value} ] ;   
        bf:heldBy <{item.adminMetadata.assigner}> ;
        bf:itemOf <{itemOf}> ;
        bf:shelfMark [ a bf:ShelfMarkDdc ;
            rdfs:label "{item.call}" ;
            bf:assigner <{item.adminMetadata.assigner}> ] ;
        bf:sublocation [ a bf:Sublocation ;
            rdfs:label "{item.call}" ] ;   
        }} }}
        """
    return graph

In [53]:
collectionUpdate = FusekiUpdate('http://localhost:3030', 'collection')

In [54]:
item = request.items[0]
graph = MakeGraphItems(item, "https://bibliokeia.com/resources/instance/bkc-2")

responseDelete = collectionUpdate.run_sparql(graph)
responseDelete.convert()


{'statusCode': 200, 'message': 'Update succeeded'}

In [56]:
request.itemOf

'https://bibliokeia.com/resources/instance/bkc-2'

In [57]:
upInstance = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        INSERT DATA
        {{ GRAPH <{request.itemOf}> {{ <{request.itemOf}>  bf:hasItem  <https://bibliokeia.com/resources/item/23-8486> }} }}"""

responseDelete = collectionUpdate.run_sparql(upInstance)
responseDelete.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [58]:
from pysolr import Solr

In [77]:
solr = Solr('http://localhost:8983/solr/acervo/', timeout=10)

In [67]:
item = request.items[0]

In [80]:
instanceID = request.itemOf.uri.split("/")[-1]
doc = {
        "id": "1",
        "type": "Item",
        "heldBy": "Biblioteca do INPA",
        "call": item.call,
        "shelf": item.shelf, 
        "barcode": item.barcode,
        "itemOf": {'id': f"1/itemOf/{instanceID}",
                   'mainTitle': request.itemOf.label,
                   'uri': request.itemOf.uri}
        }
solr.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":147}}\n'

In [ ]:
{
  "id": "P11!S21",
  "_root_": "P11!prod",
  "manuals": { "add": { "id": "P11!D99",
                        "name_s": "Why Red Staplers Are the Best",
                        "content_t": "Once upon a time, Mike Judge ...",
                      } }
} 

In [82]:
instance = {
    'id': 'bkc-2',
    'hasItem': {  "add": { 'id': f"bkc-2/hasItem/i",
                          'uri': "url",
                          'barcode': "barcode"    }
    }
}
solr.add([instance], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":106}}\n'

In [85]:
request.items[0]

Item(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='2023-06-27', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 27, 16, 0, 52, 826753), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='23-8486')], status=Status(value='mstatus:new', label='novo')), call='582.56 O48p 2023', shelf='E11-P05', barcode='23-8486')